In [1]:
import pandas as pd
import numpy as np

In [14]:
result_path_u = 'E:/泰迪杯/数据挖掘2021/模糊综合评价结果/模糊综合评价酒店评分utf-8.csv'
result_path_g = 'E:/泰迪杯/数据挖掘2021/模糊综合评价结果/模糊综合评价酒店评分gbk.csv'

In [15]:
col = ['序号','酒店名称','总得分','服务得分','位置得分','设施得分','卫生得分','性价比得分']
rating = pd.DataFrame(columns=col)
for doc_num in range(1,51):
    # 读入数据
    review_path = 'E:/泰迪杯/数据挖掘2021/情感分析结果utf-8/H' + str(doc_num) + 'utf-8.csv'
    reviews = pd.read_csv(review_path, encoding='utf-8')[['评价方面','情感等级']]
    col = {'评价方面':'topic','情感等级':'emotion_level'}
    reviews.rename(columns=col,inplace=True)
    
    # 行名是主题，列名是情感等级，数值是频数
    R_tmp = np.empty(shape=(0,5))
    order = ['服务','位置','设施','卫生','性价比']
    for emotionLevel in range(1,6):
        # 情感等级为emotionLevel的数据表
        level = reviews[reviews['emotion_level'] == emotionLevel]
    
        # 提取在该情感等级下每个评价方面的数量，为向量
        num_arr = level['topic'].value_counts()
        if len(num_arr)>0:
            num_arr = num_arr.reindex(order) # 统一主题顺序为order
            num_arr = num_arr.values
        if len(num_arr)==0: # 向量长度为0
            num_arr = np.zeros((1,5))
    
        # 将向量合并到R中
        R_tmp = np.vstack((R_tmp, num_arr))
    R_tmp = R_tmp.T
    np.nan_to_num(R_tmp, copy=False)  # 将缺失值填充为0
    
    # 计算主题的评价次数
    c = np.array([1,1,1,1,1]).reshape(5,1)
    topic_sum = R_tmp.dot(c)
    
    # 计算评判矩阵R
    R = R_tmp/topic_sum
    
    # 计算各评价方面的权重矩阵A
    A = (topic_sum/sum(topic_sum)).T
    
    # 计算模糊综合评价集B
    B = A.dot(R)
    
    df = {'序号':doc_num, '酒店名称':'H'+str(doc_num)}
    level = np.array((5,4,3,2,1))
    df['总得分'] = np.round(level.dot(B.T),1)[0]    # 计算总体满意度
    df['服务得分'] = np.round(level.dot(R[0].T),1)  # 计算服务满意度
    df['位置得分'] = np.round(level.dot(R[1].T),1)  # 计算位置满意度
    df['设施得分'] = np.round(level.dot(R[2].T),1)  # 计算设施满意度
    df['卫生得分'] = np.round(level.dot(R[3].T),1)  # 计算卫生满意度
    df['性价比得分'] = np.round(level.dot(R[4].T),1)  # 计算性价比满意度
    
    rating = rating.append(df,ignore_index=True)

In [16]:
rating

,序号,酒店名称,总得分,服务得分,位置得分,设施得分,卫生得分,性价比得分
0,1,H1,3.2,3.1,3.3,3.3,3.1,3.2
1,2,H2,3.0,2.9,3.0,3.1,3.0,3.1
2,3,H3,3.1,3.1,3.0,3.2,3.0,3.2
3,4,H4,3.2,3.0,3.3,3.3,3.0,3.3
4,5,H5,3.1,2.9,3.1,3.3,2.9,3.1
5,6,H6,3.0,3.1,3.0,3.0,2.8,3.2
6,7,H7,3.2,3.1,3.2,3.1,3.0,3.2
7,8,H8,3.0,2.7,3.1,3.2,2.9,3.1
8,9,H9,3.1,3.0,3.1,3.6,2.7,3.3
9,10,H10,3.2,3.0,3.1,3.5,3.2,3.1


In [17]:
rating.to_csv(result_path_u, encoding='utf-8', index=False)
rating.to_csv(result_path_g, encoding='gbk', index=False)

In [18]:
R

array([[0.        , 0.08333333, 0.25      , 0.41666667, 0.25      ],
       [0.16666667, 0.01851852, 0.5       , 0.22222222, 0.09259259],
       [0.22222222, 0.08888889, 0.4       , 0.17777778, 0.11111111],
       [0.        , 0.07142857, 0.28571429, 0.42857143, 0.21428571],
       [0.09565217, 0.0173913 , 0.53043478, 0.24347826, 0.11304348]])

In [19]:
A

array([[0.05      , 0.225     , 0.1875    , 0.05833333, 0.47916667]])

In [20]:
B

array([[0.125     , 0.0375    , 0.47083333, 0.24583333, 0.12083333]])